---
title: "Fourrier Transformation"
subtitle: ""
date: last-modified
sidebar: auto
number-sections: false
toc: true
author:
  - Jumbong Junior 

categories: []
tags: [""]
title-block-banner: false
bibliography: references.bib
format: 
  html: 
    mainfont: Times New Roman
    fontsize: 1.1em

jupyter: python3
notice: |
    @balac2011transformee @cooley1965algorithm
---




# Introduction

The primary goal of the Fourier transformation is to analyze signals by decomposing them into their constituent frequencies. It was Joseph Fourier who first recognized the significance of spectral decomposition of a signal. Indeed, he demonstrated that any periodic signal can be decomposed into a finite sum of sinusoidal signals with constant frequencies and amplitudes. The finite set of these frequencies constitutes the spectrum of the signal. 

Beyond spectral analysis, the Fourier transformation is employed in solving partial differential equations, evaluating integrals, and computing sums of series. It has applications in various fields, including physics, engineering, and signal processing. In recent years, it has found new uses in finance, such as estimating financial asset volatility, which We will cover in future posts.

In this article, we explore the numerical computation of the Fourier transform for both real and complex functions. Drawing on the work of @balac2011transformee, who proposed a quadrature-based approach, we illustrate how this method can be applied. We also demonstrate how the Fast Fourier Transform (FFT) algorithm [ for more details see @cooley1965algorithm] enables efficient computation of the discrete Fourier transform, making it well-suited for calculating the Fourier transform of integrable functions and the Fourier coefficients of periodic functions.

This article was motivated by the realization that the numerical methods implemented in Python—such as [`scipy.fft` and `numpy.fft`] do not directly compute the Fourier transform of a function. This observation had already been made by @balac2011transformee in the context of MATLAB.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift

# Define the function f(t) = exp(-pi * t^2)
def f(t):
    return np.exp(-np.pi * t**2)

# Parameters
N = 1024
T = 1.0 / 64
t = np.linspace(-N/2*T, N/2*T, N, endpoint=False)
y = f(t)

# FFT with scipy
yf_scipy = fftshift(fft(y)) * T
xf = fftshift(fftfreq(N, T))
FT_exact = np.exp(-np.pi * xf**2)

# FFT with numpy
yf_numpy = np.fft.fftshift(np.fft.fft(y)) * T
xf_numpy = np.fft.fftshift(np.fft.fftfreq(N, T))

# Plot side-by-side
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# Left: Scipy FFT
axs[0].plot(xf, FT_exact, 'k-', linewidth=1.5, label='Exact FT')
axs[0].plot(xf, np.real(yf_scipy), 'r--', linewidth=1, label='FFT (scipy)')
axs[0].set_xlim(-6, 6)
axs[0].set_ylim(-1, 1)
axs[0].set_title("Scipy FFT")
axs[0].set_xlabel("Frequency")
axs[0].set_ylabel("Amplitude")
axs[0].legend()
axs[0].grid(True)

# Right: Numpy FFT
axs[1].plot(xf_numpy, FT_exact, 'k-', linewidth=1.5, label='Exact FT')
axs[1].plot(xf_numpy, np.real(yf_numpy), 'b--', linewidth=1, label='FFT (numpy)')
axs[1].set_xlim(-6, 6)
axs[1].set_title("NumPy FFT")
axs[1].set_xlabel("Frequency")
axs[1].legend()
axs[1].grid(True)

plt.suptitle("Comparison of FFT Implementations vs. Exact Fourier Transform", fontsize=14)
plt.tight_layout()
plt.show()

## Definition and Properties of the Fourier Transform

We must first define the space in which the function we want to compute the Fourier transform is defined and the caracteristics they must satisfy so that the Fourier transform exists.
These functions are defined in the space of integrable functions, denoted by $L^1(\mathbb{R,K})$ which are functions $f:\mathbb{R} \to \mathbb{K}$ [where $\mathbb{K}$ is either $\mathbb{R}$ or $\mathbb{C}$]. These functions are integrable in the sense of Lebesgue, meaning that the integral of their absolute value is finite:
$$
\int_{-\infty}^{+\infty} |f(t)| dt < +\infty.
$$

So for a function $f$ to be in $L^1(\mathbb{R,K})$, the product $f(t) \cdot e^{-2i\pi\nu t}$ is integrable for all $\nu \in \mathbb{R}$. And for all $\nu \in \mathbb{R}$, the function $\hat{f}$ [notée egalement $F(f)$] defined by 
$$
\hat{f}(\nu) = \int_{-\infty}^{+\infty}
f(x) e^{-2i\pi\nu t} dt
$$

is well-defined and is called the Fourier transform of $f$. 

We deduce in this formula that a Fourier transform is a complex-valued function, and it is a linear operator. We can also deduce others properties of the Fourier transform in particular for a function $f$ in $L^1(\mathbb{R,R})$, une fonction à valeurs réelles :

- Translation property: If $f$ is in $L^1(\mathbb{R,K})$, $t_0 \in \mathbb{R}$,  and $h : t \in \mathbb{R} \mapsto f(t - t_0)$, we have $h \in L^1(\mathbb{R,K})$ and $\hat{h}(\nu) = e^{-2i\pi\nu t_0} \hat{f}(\nu)$.
- Homothety property: If $f$ is in $L^1(\mathbb{R,K})$ and $a \in \mathbb{R}$, then the function $g : t \in \mathbb{R} \mapsto f(at)$ is in $L^1(\mathbb{R,K})$ and $\hat{g}(\nu) = \frac{1}{|a|} \hat{f}\left(\frac{\nu}{a}\right)$.

- Modulation property: If $f$ is in $L^1(\mathbb{R,K})$ and $\nu_0 \in \mathbb{R}$,  and $h : t \in \mathbb{R} \mapsto f(t) e^{2i\pi\nu_0 t}$, we have $h \in L^1(\mathbb{R,K})$ and $\hat{h}(\nu) = \hat{f}(\nu - \nu_0)$.

- If $f \in L^1(\mathbb{R,R})$ and est pair, then $\hat{f}$ is also real-valued and the real part of $\hat{f}$ is also an even function.
- If $f$ is in $L^1(\mathbb{R,R})$ and is odd, then $\hat{f}$ is est une fonction imaginaire pure and the imaginary part of $\hat{f}$ is also an odd function.

For some functions, the Fourier transform can be computed analytically. For example, for the function $f : t \in \mathbb{R} \mapsto \mathbb{1}_{[-\frac{a}{2}, \frac{a}{2}]}(t)$, the Fourier transform is given by:
$$
\hat{f} : \nu \in \mathbb{R} \mapsto a sinc(a \pi \nu)
$$
where $sinc(t) = \frac{\sin(t)}{t}$ is the sinc function.

However, for many functions, the Fourier transform cannot be computed analytically. In such cases, we can use numerical methods to approximate the Fourier transform. We will explore these numerical dans la suite of this article.

## How to approximate the Fourier Transform?

The Fourier transform of a function $f$ is defined as an integral over the entire real line. However, for the functions that are integral in the sense of lebesgue and that have a practical applications tend to 0 as $|t| \to +\infty$. And we can approximate the Fourier transform by integrating over a finite interval $[-T, T]$. If the lenght of the interval is large enough, or if the function decays quickly when t tends to infinity, this approximation will be accurate.

$$
\hat{f}(\nu) \approx \int_{-T}^{T} f(t) e^{-2i\pi\nu t} dt
$$

In his article, @balac2011transformee va plus loin en montrant qu'approximating the Fourier transform met en oeuvre trois outils mathématiques :

- Les séries de Fourier dans le cadre d'un signal périodique.
- La transformée de Fourier dans le cadre d'un signal non périodique.
- La transformée de Fourier discrète dans le cadre d'un signal discret.

And for each of these tools, computing the Fourier transform revient à calculer l'intgrale below:
$$
\hat{f}(\nu) \approx \int_{-\frac{T}{2}}^{\frac{T}{2}} f(t) e^{-2i\pi\nu t} dt
$$
I recommends reading his article for more details on these three tools. Then we will focus on the numerical computation of the Fourier transform using quadrature methods, which is a numerical integration technique.

## Numerical Computation of the Fourier Transform

We show that to compute the Fourier transform of a function $f$ consists to approximating it by the integral below in the interval $[-\frac{T}{2}, \frac{T}{2}]$:
$$
\underbrace{
  \int_{-\infty}^{+\infty} f(t)\, e^{-2\pi i \nu t} \, dt
}_{=\,\hat{f}(\nu)}
\approx
\underbrace{
  \int_{-\frac{T}{2}}^{\frac{T}{2}} f(t)\, e^{-2\pi i \nu t} \, dt
}_{=\,\tilde{S}(\nu)}
$$

where T is a large enough number such that the integral converges. Une valeur approchée of the integral $\tilde{S}(\nu) = \int_{-\frac{T}{2}}^{\frac{T}{2}} f(t)\, e^{-2\pi i \nu t} \, dt$ can be computed using quadrature methods. In the next section, we will approximate the integral using the quadrature method of rectangles à gauche.

## Quadrature Method of Rectangles à Gauche

For computing the integral $\tilde{S}(\nu) = \int_{-\frac{T}{2}}^{\frac{T}{2}} f(t)\, e^{-2\pi i \nu t} \, dt$, using the quadrature method of rectangles à gauche, we must respect the following steps:
1. **Discretization of the Interval**: We discretize the interval $[-\frac{T}{2}, \frac{T}{2}]$ into $N$ uniform subintervals of length $h_t = \frac{T}{N}$. The points in the interval are given by:

$$
t_k = -\frac{T}{2} + k \cdot h_t, \quad k = 0, 1, \ldots, N-1.
$$

2. **Approximation of the Integral**: Using the Chasles relation, we can approximate the integral $\tilde{S}(\nu)$ as follows:

$$
\tilde{S}(\nu) = \int_{-\frac{T}{2}}^{\frac{T}{2}} f(t)\, e^{-2\pi i \nu t} \, dt = \sum_{k=0}^{N-1}  \int_{t_k}^{t_{k+1}} f(t)\, e^{-2\pi i \nu t} \, dt.
$$

By taking into account that we have $t_{k+1} - t_k = h_t$, and $t_k = -\frac{T}{2} + k \cdot h_t = T(\frac{k}{N} - \frac{1}{2})$, we can rewrite the integral as:
$$
\tilde{S}(\nu) = \sum_{k=0}^{N-1} f(t_k) e^{-2\pi i \nu t_k} h_t.
$$
3. **Final Formula**: The final formula for the approximation of the Fourier transform is given by:

$$
\boxed{
\forall \nu \in \mathbb{R} \quad
\underbrace{
\int_{-\frac{T}{2}}^{\frac{T}{2}} f(t)\, e^{-2\pi i \nu t} \, dt
}_{=\,\tilde{S}(\nu)}
\approx
\underbrace{
\frac{T}{n} e^{i \pi \nu T} \sum_{k=0}^{n-1} f_k\, e^{-2 i \pi \nu T k / n}
}_{=\,\tilde{S}_n(\nu)}
\quad \text{où } f_k = f\left( \frac{2k - n}{2n} T \right).
}
$$

### Mis en Oeuvre in Python 

The function `tfquad` below implements the quadrature method of rectangles à gauche to compute the Fourier transform of a function `f` at a given frequency `nu`. 


In [ ]:
import numpy as np

def tfquad(f, nu, n, T):
    """
    Computes the Fourier transform of a function f at frequency nu
    using left Riemann sum quadrature over the interval [-T/2, T/2].

    Parameters:
    ----------
    f : callable
        The function to transform. Must accept a NumPy array as input.
    nu : float
        The frequency at which to evaluate the Fourier transform.
    n : int
        Number of quadrature points.
    T : float
        Width of the time window [-T/2, T/2].

    Returns:
    -------
    tfnu : complex
        Approximated value of the Fourier transform at frequency nu.
    """
    k = np.arange(n)
    t_k = (k / n - 0.5) * T
    weights = np.exp(-2j * np.pi * nu * T * k / n)
    prefactor = (T / n) * np.exp(1j * np.pi * nu * T)

    return prefactor * np.sum(f(t_k) * weights)

We can also use the function scipy's `quad` functionnfor defining the Fourier transform of a function `f` at a given frequency `nu`.  The function `tf_integral` below implements this approach. It uses numerical integration to compute the Fourier transform of a function `f` over the interval $[-T/2, T/2]$.


In [ ]:
from scipy.integrate import quad

def tf_integral(f, nu, T):
    """Compute FT of f at frequency nu over [-T/2, T/2] using scipy quad."""
    real_part = quad(lambda t: np.real(f(t) * np.exp(-2j * np.pi * nu * t)), -T/2, T/2)[0]
    imag_part = quad(lambda t: np.imag(f(t) * np.exp(-2j * np.pi * nu * t)), -T/2, T/2)[0]
    return real_part + 1j * imag_part

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ----- Function Definitions -----

def f(t):
    """Indicator function on [-1, 1]."""
    return np.where(np.abs(t) <= 1, 1.0, 0.0)

def exact_fourier_transform(nu):
    """Analytical FT of the indicator function over [-1, 1]."""
    # f̂(ν) = ∫_{-1}^{1} e^{-2πiνt} dt = 2 * sinc(2ν)
    return 2 * np.sinc(2 * nu)


# ----- Computation -----

T = 2.0
n = 32
nu_vals = np.linspace(-6, 6, 500)
exact_vals = exact_fourier_transform(nu_vals)
approx_vals = np.array([tfquad(f, nu, n, T) for nu in nu_vals])

In [ ]:
fig, axs = plt.subplot_mosaic([["quad", "tfquad"]], figsize=(12, 4), layout="constrained")

# Plot using scipy.integrate.quad
axs["quad"].plot(nu_vals, np.real(integral_vals), 'b-', linewidth=2, label=r'$\hat{f}$ (quad)')
axs["quad"].plot(nu_vals, np.real(approx_vals), 'r--', linewidth=1.5, label=r'approximation $\hat{S}_n$')
axs["quad"].set_title("TF avec scipy.integrate.quad")
axs["quad"].set_xlabel(r'$\nu$')
axs["quad"].set_ylabel('Amplitude')
axs["quad"].grid(False)
axs["quad"].legend()
axs["quad"].set_ylim(-0.5, 2.1)

# Plot using tfquad implementation
axs["tfquad"].plot(nu_vals, np.real(exact_vals), 'b-', linewidth=2, label=r'$\hat{f}$ (exact)')
axs["tfquad"].plot(nu_vals, np.real(approx_vals), 'r--', linewidth=1.5, label=r'approximation $\hat{S}_n$')
axs["tfquad"].set_title("TF avec tfquad (rectangles)")
axs["tfquad"].set_xlabel(r'$\nu$')
axs["tfquad"].grid(False)
axs["tfquad"].legend()
axs["tfquad"].set_ylim(-0.5, 2.1)

plt.show()
